# Imports and Defaults

In [1]:
import json
import os
import zipfile

import bridgestan as bs
from cmdstanpy import CmdStanModel
import numpy as np

In [2]:
seed = 1234
rng = np.random.default_rng(seed)

num_chains = 100
num_samples = 1000000

model_name = "stochastic_volatility"
model_path = model_name + ".stan"
data_path = model_name + ".data.json"

# Generate Ref Draws

In [3]:
model = CmdStanModel(stan_file=model_path)
nuts_output = model.sample(
    data=data_path,
    iter_sampling=num_samples,
    chains=num_chains,
    seed=seed,
    thin=50,
    adapt_delta=0.95,
)

ref_draws_np = nuts_output.draws()[:, :, 7:] # [draws, chains, params]
# convert dimension order to [chains, draws, params]
ref_draws_np = np.transpose(ref_draws_np, (1, 0, 2))

11:02:27 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

...

chain 98 |          | 00:00 Status

chain 99 |          | 00:00 Status

chain 100 |          | 00:00 Status

13:14:21 - cmdstanpy - INFO - CmdStan done processing.


13:14:21 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: lognormal_lpdf: Scale parameter is inf, but must be positive finite! (in 'stochastic_volatility.stan', line 15, column 4 to column 56)
	Exception: lognormal_lpdf: Scale parameter is inf, but must be positive finite! (in 'stochastic_volatility.stan', line 15, column 4 to column 56)
	Exception: lognormal_lpdf: Scale parameter is -nan, but must be positive finite! (in 'stochastic_volatility.stan', line 15, column 4 to column 56)
Exception: lognormal_lpdf: Scale parameter is inf, but must be positive finite! (in 'stochastic_volatility.stan', line 15, column 4 to column 56)
	Exception: lognormal_lpdf: Scale parameter is inf, but must be positive finite! (in 'stochastic_volatility.stan', line 15, column 4 to column 56)
	Exception: lognormal_lpdf: Scale parameter is inf, but must be positive finite! (in 'stochastic_volatility.stan', line 15, column 4 to column 56)
	Exception: lognormal_lpdf: Scale parameter is inf, b

In [5]:
summary = nuts_output.summary()

In [ ]:
count    5.040000e+02
mean     1.995443e+06
std      1.092657e+04
min      1.834750e+06
25%      1.993654e+06
50%      1.996816e+06
75%      1.999424e+06
max      2.008285e+06
Name: N_Eff, dtype: float64



In [10]:
summary["N_Eff"].describe()

count    5.040000e+02
mean     1.995443e+06
std      1.092657e+04
min      1.834750e+06
25%      1.993654e+06
50%      1.996816e+06
75%      1.999424e+06
max      2.008285e+06
Name: N_Eff, dtype: float64


# Save and Format Reference Draws

In [4]:
# convert from numpy 3d array to json format
num_params = ref_draws_np.shape[-1]
param_names = (
    ["mu", "phi", "sigma"] + [f"tau[{i}]" for i in range(num_params)]
)

ref_draws = []
for chain in ref_draws_np:  # each chain is [n_samples, params_dim]
    chain_dict = {}
    for name, params in zip(param_names, chain.T):
        chain_dict[name] = params.tolist()
    ref_draws.append(chain_dict)

# save to file
with zipfile.ZipFile(f"{model_name}.ref_draws.json.zip", mode="w", compression=zipfile.ZIP_DEFLATED, compresslevel=9) as zip_file: 
    dumped_JSON: str = json.dumps(ref_draws, ensure_ascii=False, indent=4)
    zip_file.writestr(f"{model_name}.ref_draws.json", data=dumped_JSON)